In [1]:
from datasets import load_from_disk
dataset_all = load_from_disk('dataset_all')

In [2]:
from transformers import AutoTokenizer
model_name = "cyberagent/open-calm-medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
CUTOFF_LENGTH = 512

def tokenize(prompt, tokenizer: AutoTokenizer, cutoff_length=CUTOFF_LENGTH):
    result = tokenizer(prompt + tokenizer.eos_token,
                       truncation=True, 
                       max_length=cutoff_length,
                       padding=True)
    return {
        'input_ids': result['input_ids'],
        'attention_mask': result['attention_mask'],
    }

def generate_prompt(data_point):
    return f"""{data_point['text']} 

### Response:
{data_point['output']}"""

In [4]:
print(generate_prompt(dataset_all[0]))

エクスペリアのGPS南北が逆になるのはデフォだったのか。 

### Response:
negative


In [5]:
VAL_SET_SIZE = 4000

train_val = dataset_all.train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42)
train_data = train_val["train"]
val_data = train_val["test"]
train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))
val_data = val_data.shuffle().map(lambda x: tokenize(generate_prompt(x), tokenizer))


Loading cached split indices for dataset at /mnt/data2/home/fujie/work/2023/finetune_llm/sentiment_example/dataset_all/cache-c53cad595e9ab484.arrow and /mnt/data2/home/fujie/work/2023/finetune_llm/sentiment_example/dataset_all/cache-fd7a09784287ab6b.arrow


Map:   0%|          | 0/300157 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [6]:
train_val["train"] = train_data
train_val["test"] = val_data

In [7]:
train_val.save_to_disk('dataset_tokenized')

Saving the dataset (0/1 shards):   0%|          | 0/300157 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4000 [00:00<?, ? examples/s]